In [26]:
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import BaseDocTemplate, SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak, Table, TableStyle, PageTemplate, Frame, Flowable
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib import utils, colors
from reportlab.lib.units import inch
from reportlab.lib.colors import Color, HexColor
 
from reportlab.lib import colors
from reportlab.platypus.flowables import KeepTogether
from reportlab.pdfbase.pdfmetrics import stringWidth
 
import json
import os
 
# Custom page size and margins
PAGE_WIDTH, PAGE_HEIGHT = A4
TOP_MARGIN = 1.7 * inch
BOTTOM_MARGIN = 0.29 * inch
LEFT_MARGIN = 0.5 * inch
RIGHT_MARGIN = 0.5 * inch
 
TITLE_TEXT = "Rare Disease Profiling"
TITLE_FONT = "Helvetica-Bold"
TITLE_FONT_SIZE = 16
 
TITLE_WIDTH = stringWidth(TITLE_TEXT, TITLE_FONT, TITLE_FONT_SIZE)
TITLE_HEIGHT = TITLE_FONT_SIZE
 
AVAILABLE_PAGE_WIDTH = PAGE_WIDTH - LEFT_MARGIN - RIGHT_MARGIN
 
TITLE_PADDING = 10
PATIENT_INFO_PADDING = 5
 
 
styles = getSampleStyleSheet()
#justified_paragraph = styles["Normal"]
#justified_paragraph.alignment = 4  # 4 corresponds to 'JUSTIFY'
 
#center_paragraph = styles["Normal"]
#center_paragraph.alignment = 1  # 1 corresponds to 'CENTER'
 
# Create a new style object for justified paragraph based on the existing Normal style
justified_paragraph = ParagraphStyle("Justified", parent=styles["Normal"])
justified_paragraph.alignment = 4  # 4 corresponds to 'JUSTIFY'
 
center_paragraph = ParagraphStyle("Centered", parent=styles["Normal"])
center_paragraph.alignment = 1  # 4 corresponds to 'JUSTIFY'
 
 
# Create a new style object for center-aligned paragraph based on the existing Normal style
imageTitle_style = ParagraphStyle("Centered", parent=styles["Normal"])
imageTitle_style.alignment = 1  # 1 corresponds to 'CENTER'
imageTitle_style.fontSize = 9
 
# Create a new style object for center-aligned paragraph based on the existing Normal style
imageCaption_style = ParagraphStyle("Centered", parent=styles["Normal"])
imageCaption_style.alignment = 1  # 1 corresponds to 'CENTER'
imageCaption_style.fontSize = 7
imageCaption_style.fontWeight = 'Normal'  # Set font weight to normal
 
# Create a new style object for center-aligned paragraph based on the existing Normal style
title_style = ParagraphStyle("Centered", parent=styles["Heading2"])
title_style.alignment = 1  # 1 corresponds to 'CENTER'
title_style.fontSize = 15
 
# Create a new style object for center-aligned paragraph based on the existing Normal style
left_paragraph = ParagraphStyle("Left", parent=styles["Normal"])
left_paragraph.alignment = 0  # 1 corresponds to 'CENTER'

right_paragraph = ParagraphStyle("Right", parent=styles["Normal"])
right_paragraph.alignment = 2  # 1 corresponds to 'CENTER'

 
def json_to_pdf(patient_metadata_json, result_json, pdf_file_path):
    # Open the JSON file
    with open(patient_metadata_json, 'r') as f:
        # Load the JSON data
        metadata = json.load(f)
    metadata = metadata['metadata']
 
    with open(result_json, 'r') as f:
        # Load the JSON data
        result = json.load(f)
    result = result['condition']
    # Patient Information
 
    patient_id_num = metadata["Patient ID"]
    patient_ethnicity = metadata["Ethnicity"]
 
    patient_id_data = [
        [Paragraph("Name:", left_paragraph), Paragraph(":  "+metadata["Name"],left_paragraph), Paragraph(f"Patient ID",left_paragraph),Paragraph(":  "+metadata["Patient ID"],left_paragraph)],
        [Paragraph("Gender", left_paragraph), Paragraph(":  "+metadata["Gender"],left_paragraph), Paragraph(f"Ethnicity",left_paragraph),Paragraph(":  "+metadata["Ethnicity"],left_paragraph)],
        [Paragraph("Date of Birth", left_paragraph), Paragraph(":  "+str(metadata["Date of Birth"]),left_paragraph), None,None],
    ]
    patient_id_table_colWidths = [100, 250, 100, 100]
    patient_id_table = Table(patient_id_data, colWidths=patient_id_table_colWidths)
    patient_id_table.setStyle([
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),  # Align images at the top of the cells
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),  # Align images to the left
        ('BOTTOMPADDING', (0, 0), (-1, -1), 2),  # Padding at the bottom of the cells
        ('TOPPADDING', (0, 0), (-1, -1), 2),  # Padding at the top of the cells
        ('LEFTPADDING', (0, 0), (-1, -1), 2),  # Padding on the left of the cells
        ('RIGHTPADDING', (0, 0), (-1, -1), 2),
    ])
   
    required_width, required_height = patient_id_table.wrap(PAGE_WIDTH, PAGE_HEIGHT)
   
    def myFirstPage(canvas, doc, table_data=patient_id_table):
        canvas.saveState()
        # Add the title for the first page
        canvas.setFont(TITLE_FONT, TITLE_FONT_SIZE)
        canvas.drawString((PAGE_WIDTH/2)-(TITLE_WIDTH/2), PAGE_HEIGHT-TOP_MARGIN-TITLE_HEIGHT, TITLE_TEXT)
        canvas.restoreState()
 
        table = table_data
       
        # Before positioning and drawing the table, calculate its required size
        required_width, required_height = table.wrap(PAGE_WIDTH, PAGE_HEIGHT)
 
        table_x_position = LEFT_MARGIN
        table_y_position = PAGE_HEIGHT - TOP_MARGIN - required_height - TITLE_HEIGHT - TITLE_PADDING # Adjust based on the table's height
 
        table.wrapOn(canvas, *doc.pagesize)
        table.drawOn(canvas, table_x_position, table_y_position)  # Adjust the coordinates as needed
 
    def myLaterPages(canvas, doc, table_data=patient_id_table):
        canvas.saveState()
        table = table_data
 
        required_width, required_height = table.wrap(PAGE_WIDTH, PAGE_HEIGHT)
        globals()['required_height'] = required_height
 
        table_x_position = LEFT_MARGIN
        table_y_position = PAGE_HEIGHT - TOP_MARGIN - required_height  # Adjust based on the table's height
 
        table.wrapOn(canvas, *doc.pagesize)
        table.drawOn(canvas, table_x_position, table_y_position)  # Adjust the coordinates as needed
 
        canvas.restoreState()
 
 
    # Create a PDF document with ReportLab
    doc = SimpleDocTemplate(pdf_file_path, pagesize=A4, leftMargin=LEFT_MARGIN, rightMargin=RIGHT_MARGIN, topMargin=TOP_MARGIN+required_height+PATIENT_INFO_PADDING, bottomMargin=BOTTOM_MARGIN)
    story = []
 
 
    styles = getSampleStyleSheet()
 
 
    story.append(Spacer(1, TITLE_HEIGHT))
    story.append(Spacer(1, TITLE_PADDING))
    story.append(Spacer(1, PATIENT_INFO_PADDING))
    story.append(Paragraph("Family History", styles['Heading3']))
    story.append(Paragraph(metadata["Family History"], justified_paragraph))
    story.append(Paragraph("<br/>", styles['Normal']))  # Add some spacing
    story.append(Paragraph("Listed of Disease/Condition in this Report", styles['Heading3']))
    for condition, variants in result.items():
        story.append(Paragraph(f'---  {condition}', justified_paragraph))
    story.append(PageBreak())
 
 
    story.append(Paragraph("Report Disclaimer", styles['Heading3']))
    story.append(Paragraph('''
                            Lorem ipsum dolor sit amet, consectetur adipiscing elit. Mauris vehicula libero at sollicitudin dignissim. Nam aliquam arcu ut risus tempus facilisis. Etiam congue tempus tellus. Nulla eu enim a mi sagittis sagittis nec bibendum quam. Proin eget pulvinar tortor. Fusce nibh purus, tempus vitae hendrerit id, pretium sit amet eros. Cras eget dui justo. Ut interdum pulvinar facilisis. Quisque luctus efficitur nisi eu imperdiet. Aliquam fermentum enim non elit aliquet eleifend. Sed nunc sem, tempor in venenatis in, volutpat non magna. Suspendisse sodales, purus nec imperdiet molestie, tortor magna tristique nisi, id pulvinar eros lorem quis lacus. Cras eu augue lorem.
 
                            Pellentesque at turpis eget nunc sollicitudin semper sed at dolor. Nulla tristique maximus tellus at finibus. Vestibulum eu erat sit amet enim tristique condimentum at sed erat. Ut massa quam, euismod non dictum ut, vehicula quis quam. Nullam id lorem maximus, imperdiet neque eget, tristique mauris. Vivamus accumsan mollis lacus et scelerisque. Suspendisse elementum ex sed dui interdum gravida. Fusce vel risus orci. Fusce efficitur, odio id luctus placerat, metus orci sollicitudin mauris, a sagittis urna nulla facilisis nibh. Maecenas luctus semper libero, quis lobortis ligula placerat eu. Sed commodo orci lectus, a luctus tellus lobortis a.
 
                            Duis sit amet tempus metus. Proin semper ac leo et aliquet. Vestibulum porttitor et orci ut commodo. Praesent sed bibendum arcu. Integer rhoncus magna interdum, rutrum augue vel, rutrum turpis. Nulla ut nisi ornare, viverra nisi ut, egestas ipsum. Nam porttitor risus id lorem elementum porttitor. Curabitur sollicitudin metus non euismod ullamcorper. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Phasellus neque nulla, congue sed consequat ac, elementum id nulla. Mauris nec porttitor purus. Mauris dapibus eget magna et tempus. Morbi tincidunt, sapien vel mattis blandit, elit nulla sodales mi, quis semper ante eros vitae nunc. Phasellus euismod diam id urna pulvinar condimentum id non lacus. Ut pretium facilisis tortor.
 
                            Sed eget orci fringilla tortor faucibus luctus euismod quis velit. Suspendisse lorem diam, pellentesque sed sollicitudin at, semper vel leo. Proin id luctus risus. Donec accumsan odio quis nunc ultricies, sed tincidunt metus porta. Vestibulum interdum dui lectus, in placerat nisi tempor quis. Maecenas at erat in nunc tincidunt faucibus eget vitae mauris. Integer ultricies tristique dui non convallis. Curabitur tempor vehicula lacus, sit amet fringilla nisl efficitur a. Fusce vitae faucibus risus. Curabitur ut nulla eget sapien fermentum eleifend. Proin ante magna, pharetra ac tempor ac, lobortis in erat. Nam ipsum tortor, venenatis semper ligula ut, pharetra finibus massa. Fusce euismod vulputate tellus vitae pellentesque. Fusce sit amet tincidunt mi. Aliquam erat volutpat.
 
                            Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Mauris vel semper metus. Sed tortor metus, aliquam non eleifend at, convallis id ligula. Nam fringilla, est non mollis semper, turpis arcu iaculis neque, in sagittis libero leo ut sapien. Maecenas maximus pharetra magna non molestie. Mauris euismod fermentum varius. Donec pulvinar sit amet lorem at maximus. Donec vestibulum nibh eu varius tincidunt.
                            ''', justified_paragraph))
 
 
    story.append(PageBreak())
 
 
    # Iterate through each condition to add it to the document
    for condition, variants in result.items():
        # Add condition name as a heading
        story.append(Paragraph(condition, styles['Heading3']))
        story.append(Paragraph("<br/>", styles['Normal']))  # Add some spacing
        story.append(Paragraph('Variant Classification and Significance', styles['Heading4']))
 
        # Table data including the headers
        table_data1 = [[
            "Chromosome", "Position", "Reference \nAllele", "Alternate \nAllele",
            "Variant \nQuality", "Filter \nStatus", "ACMG \nClassification", "ClinVar \nSignificance",
        ]]
 
        # Add the details for each variant
        for variant in variants:
            row = [
                variant.get("Chromosome", ""),
                str(variant.get("Position", "")),
                variant.get("Reference Allele", ""),
                variant.get("Alternate Allele", ""),
                str(variant.get("Variant Quality", "")),
                variant.get("Filter Status", ""),
                variant.get("ACMG Classification", ""),
                variant.get("ClinVar Significance", "")
            ]
            table_data1.append(row)
 
        table1 = Table(table_data1, repeatRows=1) # colWidths=colWidths,
        table1.setStyle(TableStyle([
            ('GRID', (0,0), (-1,-1), 1, colors.black),
            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
            ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
        ]))
       
        story.append(table1)
 
        story.append(Paragraph("<br/>", styles['Normal']))  # Add some spacing
        story.append(Paragraph('Variant Population Frequency', styles['Heading4']))
        # Table data including the headers
        table_data2 = [[
            "Chromosome", "Position", "Reference \nAllele", "Alternate \nAllele",
            "Variant \nQuality", "Filter \nStatus", "gnomAD \nAF", "1000g \nAF"
        ]]
 
        # Add the details for each variant
        for variant in variants:
            row = [
                variant.get("Chromosome", ""),
                str(variant.get("Position", "")),
                variant.get("Reference Allele", ""),
                variant.get("Alternate Allele", ""),
                str(variant.get("Variant Quality", "")),
                variant.get("Filter Status", ""),
                str(variant.get("gnomAD AF", "")),
                str(variant.get("1000g AF", ""))
            ]
            table_data2.append(row)
 
        table2 = Table(table_data2, repeatRows=1) # colWidths=colWidths,
        table2.setStyle(TableStyle([
            ('GRID', (0,0), (-1,-1), 1, colors.black),
            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
            ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
        ]))
       
        story.append(table2)
        story.append(PageBreak())
 
        story.append(Paragraph(condition, styles['Heading3']))
        story.append(Paragraph("<br/>", styles['Normal']))  # Add some spacing
        story.append(Paragraph('Variant Consequence and Inheritance Pattern', styles['Heading4']))
 
        # Table data including the headers
        table_data3 = [[
            "Chromosome", "Position",
            "Reference \nAllele", "Alternate \nAllele",
            "Consequence", "Inheritance \nPattern"
        ]]
 
        # Add the details for each variant
        for variant in variants:
            row = [
                variant.get("Chromosome", ""),
                str(variant.get("Position", "")),
                variant.get("Reference Allele", ""),
                variant.get("Alternate Allele", ""),
                variant.get("Consequence", ""),
                variant.get("Inheritance Pattern", ""),
            ]
            table_data3.append(row)
 
        # Create the table for the current condition
        #colWidths = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
        table3 = Table(table_data3, repeatRows=1) # colWidths=colWidths,
        table3.setStyle(TableStyle([
            ('GRID', (0,0), (-1,-1), 1, colors.black),
            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
            ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
        ]))
       
        story.append(table3)
        story.append(PageBreak())
 
        story.append(Paragraph(condition, styles['Heading3']))
        story.append(Paragraph("<br/>", styles['Normal']))  # Add some spacing
        story.append(Paragraph('Variant Age of Onset and Prevalence', styles['Heading4']))
 
        # Table data including the headers
        table_data4 = [[
            "Chromosome", "Position",
            "Reference \nAllele", "Alternate \nAllele",
            "Age of \nOnset","Prevalence"
        ]]
 
        # Add the details for each variant
        for variant in variants:
            row = [
                variant.get("Chromosome", ""),
                str(variant.get("Position", "")),
                variant.get("Reference Allele", ""),
                variant.get("Alternate Allele", ""),
                variant.get("Age of Onset", ""),
                variant.get("Prevalence", ""),
            ]
            table_data4.append(row)
 
        # Create the table for the current condition
        #colWidths = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
        table4 = Table(table_data4, repeatRows=1) # colWidths=colWidths,
        table4.setStyle(TableStyle([
            ('GRID', (0,0), (-1,-1), 1, colors.black),
            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
            ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
            ('BACKGROUND', (0,0), (-1,0), colors.lightgrey)
        ]))
       
        story.append(table4)
 
 
        # Add a page break after each condition, except the last
        if condition != list(result.keys())[-1]:
            story.append(PageBreak())
 
 
    # Build the PDF
    doc.build(story, onFirstPage=myFirstPage, onLaterPages=myLaterPages)

In [27]:
workdir = 'c:\\Users\\MSI\\Downloads'
metadata_path = os.path.join(workdir,'patient_data_demo.json')
result_path = os.path.join(workdir,'report_demo.json')

pdf_file_path = os.path.join(workdir,'final_report_demo.pdf')

json_to_pdf(metadata_path, result_path, pdf_file_path)